In [1]:
from collections import Counter

class positionplayers:
    def __init__(self):
        self.kickers   = None
        self.fgkickers = None
        self.punters   = None
        self.runners   = None
        self.receivers = None
        self.passers   = None
        self.defense   = None
        
    
        
    def getKickingTeam(self, playText):
        if isinstance(self.kickers, dict):
            for player, team in self.kickers.items():
                if player in playText:
                    return team
        return None
        
        
    def getFGKickingTeam(self, playText):
        if isinstance(self.fgkickers, dict):
            for player, team in self.fgkickers.items():
                if player in playText:
                    return team
        return None
        
        
    def getPuntingTeam(self, playText):
        if isinstance(self.punters, dict):
            for player, team in self.punters.items():
                if player in playText:
                    return team
        return None
        
        
    def getRunningTeam(self, playText):
            
        if isinstance(self.runners, dict):
            for player, team in self.runners.items():
                if player in playText:
                    return team
                

        run  = ("(run|Run|RUN)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")

        m = re.split("{0}\s{1}\s{2}".format(num, dist, run), playText)
        if len(m) > 1:
            runner = m[0].strip()
                
                
        return None
        
        
    def getPassingTeam(self, playText):
        ## Fix Text
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            passer = playText[:pos].strip()
        else:
            passer = playText
        
        if isinstance(self.passers, dict):
            for player, team in self.passers.items():
                if player in playText:
                    return team
        return None
        
        
    def getReceivingTeam(self, playText):
        if isinstance(self.receivers, dict):
            for player, team in self.receivers.items():
                if player in playText:
                    return team
        return None
    

class keyplayers:
    def __init__(self, fieldMap=None, driveData=None):
        self.driveData = driveData
        
        self.teams     = {}
        self.kickers   = Counter()
        self.fgkickers = Counter()
        self.punters   = Counter()
        self.runners   = Counter()
        self.receivers = Counter()
        self.passers   = Counter()
        self.defense   = Counter()
        
        self.fieldMap  = fieldMap
        self.teamsmap  = {}
        
        self.possession = None
        
        
    def getPlayers(self, lookup=None):        
        players = positionplayers()
        
        if len(self.teamsmap) != 2:
            raise ValueError("There must be two teams!: [{0}]".format(self.teamsmap))
        keys = list(self.teamsmap.keys())
        
        if lookup is not None:
            self.teamsmap[keys[0]] = lookup[keys[1]]
            self.teamsmap[keys[1]] = lookup[keys[0]]
            
            self.lookup = lookup
        else:
            self.teamsmap[keys[0]] = keys[1]
            self.teamsmap[keys[1]] = keys[0]
            
            self.lookup = {}
            self.lookup[keys[0]] = keys[0]
            self.lookup[keys[1]] = keys[1]
        
        
        print(self.passers)
        
        passerNames = [x[0] for x in self.passers.most_common()]
        passerTeams = [self.teams[x].most_common(1)[0][0] for x in passerNames]
        print(passerTeams)
        passerTeams = [self.lookup[x] for x in passerTeams]
        players.passers     = dict(zip(passerNames, passerTeams))
        print(players.passers)
        
        fgkickerNames = [x[0] for x in self.fgkickers.most_common()]
        fgkickerTeams = [self.teams[x].most_common(1)[0][0] for x in fgkickerNames]
        fgkickerTeams = [self.lookup[x] for x in fgkickerTeams]
        players.fgkickers     = dict(zip(fgkickerNames, fgkickerTeams))

        ### I need to swap the kickers because they show up with the other team in the text
        kickerNames = [x[0] for x in self.kickers.most_common()]
        kickerTeams = [self.teams[x].most_common(1)[0][0] for x in kickerNames]
        kickerTeams = [self.teamsmap[x] for x in kickerTeams]
        players.kickers     = dict(zip(kickerNames, kickerTeams))

        runnerNames = [x[0] for x in self.runners.most_common()]
        runnerTeams = [self.teams[x].most_common(1)[0][0] for x in runnerNames]
        runnerTeams = [self.lookup[x] for x in runnerTeams]
        players.runners     = dict(zip(runnerNames, runnerTeams))

        receiverNames = [x[0] for x in self.receivers.most_common()]
        receiverTeams = [self.teams[x].most_common(1)[0][0] for x in receiverNames]
        receiverTeams = [self.lookup[x] for x in receiverTeams]
        players.receivers     = dict(zip(receiverNames, receiverTeams))

        punterNames = [x[0] for x in self.punters.most_common()]
        punterTeams = [self.teams[x].most_common(1)[0][0] for x in punterNames]
        punterTeams = [self.lookup[x] for x in punterTeams]
        players.punters     = dict(zip(punterNames, punterTeams))

        defenseNames = [x[0] for x in self.defense.most_common()]
        defenseTeams = [self.teams[x].most_common(1)[0][0] for x in defenseNames]
        defenseTeams = [self.lookup[x] for x in defenseTeams]
        players.defenses     = dict(zip(defenseNames, defenseTeams))
    
        return players


        
    def setDriveData(self, driveData):
        self.driveData = driveData
    
    def setPlayText(self, playText):
        self.playText = playText

        
    ################################################################################
    ## Analyze Game
    ################################################################################
    def analyzeGame(self, debug=False):
        if self.driveData is None:
            print("No Game Data")
            return
    
        for idr,drive in enumerate(self.driveData):
            self.possession = drive['Posession'][0]
            if self.fieldMap is not None:
                try:
                    self.possession = self.fieldMap[self.possession]
                except:
                    raise ValueError("ERROR with field map: {0}, {1}".format(self.fieldMap, self.possession))
            self.teamsmap[self.possession] = None
            drivedata  = drive['Data']

            for ipl,play in enumerate(drivedata):
                playText      = play['Data']
                if debug:
                    print(idr,'\t',ipl,'\t',playText)
                self.analyzeText(playText, debug)

            if debug:
                print("")
                
    def analyzeText(self, playText, debug=False):
        ## Remove clock/quarter information
        pos = playText.find(")")
        if pos > 0 and pos < len(playText) - 2:
            playText = playText[pos+1:].strip()
            
        self.addPasser(playText, debug)
        self.addRunner(playText, debug)
        self.addReceiver(playText, debug)
        self.addKicker(playText, debug)
        self.addFGKicker(playText, debug)
        self.addPunt(playText, debug)
        self.addDefense(playText, debug)

        
    def showAdd(self, name, value, possession, text, debug=False):
        if debug:
            sumval = sum(self.teams[name][value].values())
            indval = self.teams[name][value].most_common(1)[0][1]
            frac   = round(100*indval/sumval,0)
            
            print("\t{0: <15}{1: <30} --> {2: <10}  [{3}, {4}/{5}] \t [{6}]".format(name, value, self.possession, frac, indval, sumval, text))
            
                           

    ################################################################################
    ## Add Player <-> Team
    ################################################################################
    def addPlayerTeam(self, name, value):
        if self.teams.get(name) is None:
            self.teams[name] = {}
            
        if self.teams[name].get(value) is None:
                self.teams[name][value] = Counter()
        self.teams[name][value][self.possession] += 1                           
                           
            
    ################################################################################
    ## Passers
    ################################################################################
    def addPasser(self, playText, debug=False):
        name = "passer"
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            passer = playText[:pos].strip()
            self.passers[passer] += 1
            self.addPlayerTeam(name, passer)
            self.showAdd(name, passer, self.possession, playText, debug)
            

        keys = [" sacked by ", " sacked ", " Sacked By ", " SACKED BY "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            passer = playText[:pos].strip()
            self.addPlayerTeam(name, passer)
            self.showAdd(name, passer, self.possession, playText, debug)

    
    
    ################################################################################
    ## Runners
    ################################################################################
    def addRunner(self, playText, debug=False):
        name = "runner"
        keys = [" run for ", " Run For ", " RUN FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            runner = playText[:pos].strip()
            self.addPlayerTeam(name, runner)
            self.showAdd(name, runner, self.possession, playText, debug)
        else:
            run  = ("(run|Run|RUN)")
            num  = "([+-?]\d+|\d+)"  
            dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
            m = re.split("{0}\s{1}\s{2}".format(num, dist, run), playText)
            if len(m) > 1:
                runner = m[0].strip()
                self.addPlayerTeam(name, runner)
                self.showAdd(name, runner, self.possession, playText, debug)

    
    
    ################################################################################
    ## Receivers
    ################################################################################
    def addReceiver(self, playText, debug=False):
        name = "receiver"
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            txt = playText[pos:].strip()
            txt = " ".join(txt.split()[2:])[3:].strip()
            keys = [" for ", " For ", " FOR "]
            pos = [x for x in [txt.find(x) for x in keys] if x != -1]
            if len(pos) > 0 and min(pos) > 0:
                receiver = txt[:min(pos)].strip()
                self.addPlayerTeam(name, receiver)
                self.showAdd(name, receiver, self.possession, playText, debug)

    
    
    ################################################################################
    ## Defense
    ################################################################################
    def addDefense(self, playText, debug=False):
        name = "defense"
        keys = [" sacked by ", " Sacked By ", " SACKED BY "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:            
            txt = playText[pos+11:].strip()
            keys = [" for ", " For ", " FOR "]
            pos = max([txt.find(x) for x in keys])
            if pos > 0:
                defense = txt[:pos].strip()
                self.addPlayerTeam(name, defense)
                self.showAdd(name, defense, self.possession, playText, debug)


    
    
    ################################################################################
    ## Kickers
    ################################################################################
    def addKicker(self, playText, debug=False):
        name = "kicker"
        keys = ["kickoff", "Kickoff", "KICKOFF"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            kicker = playText[:pos].strip()
            self.addPlayerTeam(name, kicker)
            self.showAdd(name, kicker, self.possession, playText, debug)

    
    
    ################################################################################
    ## Field Goal Kicker
    ################################################################################
    def addFGKicker(self, playText, debug=False):   
        name = "fgkicker"         
        if playText[-1] == ")":
            pos = playText.rfind("(")
            if pos > 0:
                txt = playText[pos+1:-1]
                keys = [" kick", " Kick", " KICK"]
                pos = max([txt.find(x) for x in keys])
                if pos > 0:
                    fgkicker = txt[:pos].strip()
                    self.addPlayerTeam(name, fgkicker)
                    self.showAdd(name, fgkicker, self.possession, playText, debug)
            
            
        kick = ("(fg|FG)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
        m = re.split("{0}\s{1}\s{2}".format(num, dist, kick), playText)
        if len(m) > 1:
            fgkicker = m[0].strip()
            self.addPlayerTeam(name, fgkicker)
            self.showAdd(name, fgkicker, self.possession, playText, debug)
            
            
        wrd1 = ("(field|Field|FIELD)")
        wrd2 = ("(goal|Goal|GOAL)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yard|Yds|yds|Yd|yd)")
        
        m = re.split("{0}\s{1}\s{2}\s{3}".format(num, dist, wrd1, wrd2), playText)
        if len(m) > 1:
            fgkicker = m[0].strip()
            self.addPlayerTeam(name, fgkicker)
            self.showAdd(name, fgkicker, self.possession, playText, debug)

    
    
    ################################################################################
    ## Punters
    ################################################################################
    def addPunt(self, playText, debug=False):
        name = "punter"         
        keys = [" punt for ", " Punt For ", " PUNT FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            punter = playText[:pos].strip()
            self.addPlayerTeam(name, punter)
            self.showAdd(name, punter, self.possession, playText, debug)


_, _ = clock("Last Run")

NameError: name 'clock' is not defined